# District Executive Dashboard Queries 

In [1]:
from data201 import db_connection, df_query
import pandas as pd
from pandas import DataFrame
from decimal import Decimal
import os
conn = db_connection('sheql_wh.ini')
cursor = conn.cursor()

### Dataframe visualization for testing purposes

In [2]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

## Get student Attendance


In [3]:
cursor.execute('DROP PROCEDURE IF EXISTS get_school_attendance')

cursor.execute(
    """
    CREATE PROCEDURE get_school_attendance(
    IN date_input VARCHAR(20)
)
    BEGIN
        SELECT 
            s.school_id,
            s.name,
            ROUND(100 * SUM(CASE WHEN saf.status = 'present' THEN 1 ELSE 0 END) / COUNT(*), 1) AS attendance_rate
        FROM student_attendance_fact saf
        JOIN school_dim s ON saf.school_id = s.school_id
        WHERE saf.date_id = TO_DAYS(STR_TO_DATE(date_input, '%Y-%m-%d'))
        GROUP BY s.school_id;
    END
    """
)

In [4]:
date_input = '2025-05-13'

cursor.callproc('get_school_attendance', (date_input,))

for result in cursor.stored_results():
    display_results(result)

,school_id,name,attendance_rate
0,1,Melton Elementary School,96.6
1,2,Singh Elementary School,96.4
2,3,Coffey Elementary School,96.8
3,4,Griffin Elementary School,97.9
4,5,Campbell Middle School,96.0
5,6,Powell Middle School,93.0
6,7,Johnson Middle School,93.2
7,8,Lewis High School,94.2
8,9,Larsen High School,94.9
9,10,Mclaughlin Special Education School,92.0


In [5]:
date_input = '2025-05-13'

cursor.callproc('get_school_attendance', (date_input,))
school_attendance = []
for result in cursor.stored_results():
    raw_rows = result.fetchall()
    for row in raw_rows:
        school_id, school_name, attendance_rate = row
        school_attendance.append((
            school_id,
            school_name,
            float(attendance_rate)  
        ))

print(school_attendance)

[(1, 'Melton Elementary School', 96.6), (2, 'Singh Elementary School', 96.4), (3, 'Coffey Elementary School', 96.8), (4, 'Griffin Elementary School', 97.9), (5, 'Campbell Middle School', 96.0), (6, 'Powell Middle School', 93.0), (7, 'Johnson Middle School', 93.2), (8, 'Lewis High School', 94.2), (9, 'Larsen High School', 94.9), (10, 'Mclaughlin Special Education School', 92.0)]


## Get attendance by grade level for a specific school

In [6]:
cursor.execute('DROP PROCEDURE IF EXISTS get_school_grade_attendance')

cursor.execute(
    """
    CREATE PROCEDURE get_school_grade_attendance(
    IN schoolId INT,
    IN date_input VARCHAR(20)
)
    BEGIN
        SELECT 
            sd.grade_level,
            ROUND(100 * SUM(CASE WHEN saf.status = 'present' THEN 1 ELSE 0 END) / COUNT(*), 1) AS attendance_rate
        FROM student_attendance_fact saf
        JOIN student_dim sd ON saf.student_id = sd.student_id
        WHERE saf.school_id = schoolId
        AND saf.date_id = TO_DAYS(STR_TO_DATE(date_input, '%Y-%m-%d'))
        GROUP BY sd.grade_level;
    END
    """
)

In [ ]:
attendance_by_grade = {}

for school_id in range(1, 11):
    cursor.callproc('get_school_grade_attendance', (school_id, '2025-05-13'))
    
    for result in cursor.stored_results():
        attendance_by_grade[school_id] = result.fetchall()

# Now attendance_by_grade holds grade-level data for all 10 schools
for sid, data in attendance_by_grade.items():
    print(f"School ID {sid} -> {data}")

School ID 1 -> [('5', Decimal('100.0')), ('2', Decimal('96.8')), ('K', Decimal('94.3')), ('1', Decimal('95.5')), ('4', Decimal('91.9')), ('3', Decimal('100.0'))]
School ID 2 -> [('1', Decimal('97.8')), ('2', Decimal('93.7')), ('4', Decimal('100.0')), ('3', Decimal('94.7')), ('5', Decimal('100.0')), ('K', Decimal('93.3'))]
School ID 3 -> [('1', Decimal('98.1')), ('K', Decimal('100.0')), ('4', Decimal('97.6')), ('2', Decimal('93.7')), ('3', Decimal('94.0')), ('5', Decimal('98.2'))]
School ID 4 -> [('3', Decimal('96.9')), ('2', Decimal('98.0')), ('4', Decimal('96.0')), ('1', Decimal('100.0')), ('5', Decimal('98.5')), ('K', Decimal('98.1'))]
School ID 5 -> [('7', Decimal('94.8')), ('8', Decimal('98.6')), ('6', Decimal('94.6'))]
School ID 6 -> [('6', Decimal('91.2')), ('8', Decimal('93.5')), ('7', Decimal('94.0'))]
School ID 7 -> [('7', Decimal('94.2')), ('8', Decimal('94.4')), ('6', Decimal('90.9'))]
School ID 8 -> [('9', Decimal('96.2')), ('12', Decimal('96.8')), ('10', Decimal('93.6')), 

In [8]:
attendance_by_grade = []

for school_id in range(1, 11):
    cursor.callproc('get_school_grade_attendance', (school_id, '2025-05-13'))
    
    for result in cursor.stored_results():
        records = result.fetchall()
        for row in records:
            grade_level, attendance_rate = row
            attendance_by_grade.append({
                'school_id': school_id,
                'grade_level': grade_level,
                'attendance_rate': float(attendance_rate)  # Convert Decimal to float
            })

# Convert to DataFrame
df = pd.DataFrame(attendance_by_grade)
df = df[['school_id', 'grade_level', 'attendance_rate']]
display(df)

,school_id,grade_level,attendance_rate
0,1,5,100.0
1,1,2,96.8
2,1,K,94.3
3,1,1,95.5
4,1,4,91.9
5,1,3,100.0
6,2,1,97.8
7,2,2,93.7
8,2,4,100.0
9,2,3,94.7


### Store the results of these queries in a python file so that we can call on cache to run the executive dashboard

In [14]:
# Construct the path
target_path = os.path.normpath(os.path.join('..', '..', 'UI', 'attendance_snapshot.py'))

# Write to the file
with open(target_path, 'w') as f:
    f.write('attendance_by_grade = ')
    f.write(repr(attendance_by_grade))  # preserves list of dicts
    f.write('\n\n')
    f.write('attendance_by_school = ')
    f.write(repr(school_attendance))



In [11]:
cursor.close()
conn.close()